# Homework 3
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories. 

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work. 

In [1]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install imbalanced-learn delayed
import imblearn

# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`. 

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - Fill the empty values with 0.


In [2]:
# If put the data(.csv) under the same folder, you could use
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('./garments_worker_productivity.csv')
df = df.drop(columns="date")
categorical_attr = ['day', 'quarter', 'department', 'team']
print("All unique elements:")
# Prints all unique elements
for col in df[categorical_attr]:
    print(df[col].unique())

# Remaps duplicated items
df = df.replace("sweing", "sewing")
df = df.replace("finishing ", "finishing")
print("All unique elements (fixed version):")
for col in df[categorical_attr]:
    print(df[col].unique())
    
satisfied=[]
# Creates the satisfied column
for i in range(0,df.shape[0]):
    if df.at[i,"actual_productivity"] >= df.at[i,"targeted_productivity"]:
        satisfied.append(1)
    else:
        satisfied.append(0)
        
df['satisfied'] = satisfied
df = df.drop(columns="actual_productivity")
df = df.drop(columns="targeted_productivity")

# Prints columns with empty values
print("Columns with empty values:", df.columns[df.isna().any()].tolist())

# Fills them with 0
df = df.fillna(0)       

All unique elements:
['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']
['sweing' 'finishing ' 'finishing']
[ 8  1 11 12  6  7  2  3  9 10  5  4]
All unique elements (fixed version):
['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']
['sewing' 'finishing']
[ 8  1 11 12  6  7  2  3  9 10  5  4]
Columns with empty values: ['wip']


## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attributes**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.

In [3]:
# Remember to continue the task with your processed data from Exercise 1
from sklearn.preprocessing import OrdinalEncoder
# For later use
df_copy = df.copy()
encoder = OrdinalEncoder()
# Encodes categorical attributes
for attr in df[categorical_attr]:
    df[attr] = encoder.fit_transform(np.asarray(df[attr]).reshape(-1,1))
df.head()

,quarter,department,day,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,0.0,1.0,3.0,7.0,26.16,1108.0,7080,98,0.0,0,0,59.0,1
1,0.0,0.0,3.0,0.0,3.94,0.0,960,0,0.0,0,0,8.0,1
2,0.0,1.0,3.0,10.0,11.41,968.0,3660,50,0.0,0,0,30.5,1
3,0.0,1.0,3.0,11.0,11.41,968.0,3660,50,0.0,0,0,30.5,1
4,0.0,1.0,3.0,5.0,25.90,1170.0,1920,50,0.0,0,0,56.0,1


In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
X_train, y_train = train.copy().drop(columns=['satisfied']), train['satisfied']
X_test, y_test = test.copy().drop(columns=['satisfied']), test['satisfied']

### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [5]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report
X_train_cat = X_train[categorical_attr]
X_test_cat = X_test[categorical_attr]
NB = CategoricalNB()
NB.fit(X_train_cat, y_train)
print(classification_report(y_test, NB.predict(X_test_cat)))

              precision    recall  f1-score   support

           0       0.43      0.16      0.23        63
           1       0.76      0.93      0.83       177

    accuracy                           0.73       240
   macro avg       0.60      0.54      0.53       240
weighted avg       0.67      0.72      0.68       240



### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [6]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
numerical_attr = ['smv', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers']
scaler = StandardScaler()
X_train_cat = X_train[numerical_attr]
X_test_cat = X_test[numerical_attr]
NB = GaussianNB()
scaler.fit(X_train_cat)
NB.fit(scaler.transform(X_train_cat), y_train)
print(classification_report(y_test, NB.predict(scaler.transform(X_test_cat))))

              precision    recall  f1-score   support

           0       0.37      0.62      0.46        63
           1       0.82      0.63      0.71       177

    accuracy                           0.62       240
   macro avg       0.60      0.62      0.59       240
weighted avg       0.70      0.62      0.65       240



## Exercies 3 - SVM Classifier (40 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attributes**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.


In [7]:
# Remember to continue the task with your processed data from Exercise 1
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
# df_copy used here since I was having issues with encoding the former data
for attr in df_copy[categorical_attr]:
    df_copy[attr] = encoder.fit_transform(np.asarray(df_copy[attr]).reshape(-1,1))

In [8]:
# Splitting data of df_copy using the same random state
train, test = train_test_split(df_copy, test_size=0.2, random_state=42)
X_train, y_train = train.copy().drop(columns=['satisfied']), train['satisfied']
X_test, y_test = test.copy().drop(columns=['satisfied']), test['satisfied']

### Exercise 3.2 - SVM with Different Kernels (20 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please 
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [9]:
# Remember to do this task with your processed data from Exercise 3.1
from sklearn.svm import SVC
from sklearn.metrics import classification_report

scaler = StandardScaler()
scaler.fit(X_train)

clf = SVC(kernel='linear')
clf.fit(scaler.transform(X_train), np.asarray(y_train))
print(classification_report(y_test, clf.predict(scaler.transform(X_test))))

clf = SVC(kernel='rbf')
clf.fit(scaler.transform(X_train), np.asarray(y_train))
print(classification_report(y_test, clf.predict(scaler.transform(X_test))))

              precision    recall  f1-score   support

           0       1.00      0.05      0.09        63
           1       0.75      1.00      0.86       177

    accuracy                           0.75       240
   macro avg       0.87      0.52      0.47       240
weighted avg       0.81      0.75      0.65       240

              precision    recall  f1-score   support

           0       0.67      0.03      0.06        63
           1       0.74      0.99      0.85       177

    accuracy                           0.74       240
   macro avg       0.70      0.51      0.46       240
weighted avg       0.72      0.74      0.64       240



### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class. 
 - Oversample the **training data**. 
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br > 
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [10]:
# Remember to do this task with your processed data from Exercise 3.1
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=21)
print("Frequency of non-oversampled data:")
print(y_train.value_counts())
X_os, y_os = ros.fit_resample(X_train, y_train)
print("Frequency of oversampled data:")
print(y_os.value_counts())
clf = SVC(kernel='linear')
clf.fit(scaler.transform(X_os), np.asarray(y_os))
print(classification_report(y_test, clf.predict(scaler.transform(X_test))))

clf = SVC(kernel='rbf')
clf.fit(scaler.transform(X_os), np.asarray(y_os))
print(classification_report(y_test, clf.predict(scaler.transform(X_test))))

Frequency of non-oversampled data:
1    698
0    259
Name: satisfied, dtype: int64
Frequency of oversampled data:
1    698
0    698
Name: satisfied, dtype: int64
              precision    recall  f1-score   support

           0       0.42      0.62      0.50        63
           1       0.84      0.69      0.76       177

    accuracy                           0.68       240
   macro avg       0.63      0.66      0.63       240
weighted avg       0.73      0.68      0.69       240

              precision    recall  f1-score   support

           0       0.43      0.68      0.52        63
           1       0.86      0.67      0.75       177

    accuracy                           0.68       240
   macro avg       0.64      0.68      0.64       240
weighted avg       0.74      0.68      0.69       240

